![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTK4gQ9nhwHHaSXMHpeggWg7twwMCgb877smkRmtkmDeDoGF9Z6&usqp=CAU)

# <font color='PURPLE'>Ciência dos Dados na Prática</font>

# <font color='GREY'> Conhecendo o Light_GBM</font>


![alt text](https://lightgbm.readthedocs.io/en/latest/_images/LightGBM_logo_black_text.svg) 

https://lightgbm.readthedocs.io/en/latest/

### Bem-vindo à documentação do LightGBM!

LightGBM é uma estrutura de aumento de gradiente que usa algoritmos de aprendizagem baseados em árvore. Ele é projetado para ser distribuído e eficiente com as seguintes vantagens:

* Maior velocidade de treinamento e maior eficiência.

* Menor uso de memória.

* Melhor precisão.

* Suporte de aprendizagem paralela e GPU.

* Capaz de lidar com dados em grande escala.

![alt text](https://miro.medium.com/fit/c/184/184/0*CszCU2p5Ml6WbnH3.gif) 

**1. O que é Light GBM?**
Light GBM é uma estrutura de aumento de gradiente rápida, distribuída e de alto desempenho baseada no algoritmo da árvore de decisão, usada para classificação, classificação e muitas outras tarefas de aprendizado de máquina.

Uma vez que é baseado em algoritmos de árvore de decisão, ele divide a árvore em termos de folha com o melhor ajuste, enquanto outros algoritmos de reforço dividem a árvore em profundidade ou nível, em vez de em folha. Portanto, ao crescer na mesma folha no Light GBM, o algoritmo de folha inteligente pode reduzir mais perdas do que o algoritmo de nível e, portanto, resulta em uma precisão muito melhor que raramente pode ser alcançada por qualquer um dos algoritmos de aumento existentes. Além disso, é surpreendentemente muito rápido, daí a palavra 'Light'.

**2. Hyperparâmetros Importantes**

task : default value = train ; options = train , prediction ; Specifies the task we wish to perform which is either train or prediction.
application: default=regression, type=enum, options= options :
regression : perform regression task
binary : Binary classification
multiclass: Multiclass Classification
lambdarank : lambdarank application
data: type=string; training data , LightGBM will train from this data
num_iterations: number of boosting iterations to be performed ; default=100; type=int
num_leaves : number of leaves in one tree ; default = 31 ; type =int
device : default= cpu ; options = gpu,cpu. Device on which we want to train our model. Choose GPU for faster training.
max_depth: Specify the max depth to which tree will grow. This parameter is used to deal with overfitting.
min_data_in_leaf: Min number of data in one leaf.
feature_fraction: default=1 ; specifies the fraction of features to be taken for each iteration
bagging_fraction: default=1 ; specifies the fraction of data to be used for each iteration and is generally used to speed up the training and avoid overfitting.
min_gain_to_split: default=.1 ; min gain to perform splitting
max_bin : max number of bins to bucket the feature values.
min_data_in_bin : min number of data in one bin
num_threads: default=OpenMP_default, type=int ;Number of threads for Light GBM.
label : type=string ; specify the label column
categorical_feature : type=string ; specify the categorical features we want to use for training our model
num_class: default=1 ; type=int ; used only for multi-class classification

#1° Problema de Negócio 
Precisamos construir uma Máquina Preditiva que, a partir de dados históricos da empresa, faça a previsão da renda dos seus colaboradores.

## Fonte de Dados: UCI - Adults
http://archive.ics.uci.edu/ml/datasets/Adult

# 2° Análise Exploratória dos Dados

In [ ]:
#Comando de Instalação do Algoritmo
!pip install lightgbm

In [ ]:
#Comando de Instalação do Algoritmo
!pip install xgboost

In [ ]:
#importing standard libraries 
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 

#import lightgbm and xgboost 
import lightgbm as lgb 
import xgboost as xgb 

# Importação dos dados a serem utilizados para a criação do modelo de ML
data = pd.read_csv('https://raw.githubusercontent.com/llSourcell/Best-Programming-Languages-for-Machine-Learning/master/adults.txt', sep=',')
data



In [52]:
#Assigning names to the columns 
data.columns=['age','workclass','fnlwgt','education','education-num','marital_Status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','Income'] 

#glimpse of the dataset 
data.head() 



,age,workclass,fnlwgt,education,education-num,marital_Status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   final_weight    32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


#3° Pré-Processamento dos Dados

In [101]:
# Label Encoding our target variable
# Faremos a transformação do domínio existente na Variável Label(TARGET/ variável a ser predita/"Y") de caracteres para números. Para isso usamos o labelEncoder 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le=LabelEncoder() 
le.fit(data.salary) 

le.classes_ 
data.Income=Series(le.transform(data.salary))  #label encoding our target variable 
data.Income.value_counts() 

 

0    24720
1     7841
dtype: int64

In [ ]:
#One Hot Encoding of the Categorical features 
one_hot_workclass=pd.get_dummies(data.workclass) 
one_hot_education=pd.get_dummies(data.education) 
one_hot_marital_Status=pd.get_dummies(data.marital_status) 
one_hot_occupation=pd.get_dummies(data.occupation)
one_hot_relationship=pd.get_dummies(data.relationship) 
one_hot_race=pd.get_dummies(data.race) 
one_hot_sex=pd.get_dummies(data.sex) 
one_hot_native_country=pd.get_dummies(data.native_country) 

#removing categorical features 
data.drop(['workclass','education','marital_Status','occupation','relationship','race','sex','native_country'],axis=1,inplace=True) 

 

In [ ]:
#Merging one hot encoded features with our dataset 'data' 
data=pd.concat([data,one_hot_workclass,one_hot_education,one_hot_marital_Status,one_hot_occupation,one_hot_relationship,one_hot_race,one_hot_sex,one_hot_native_country],axis=1) 
data



In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Columns: 219 entries, age to Yugoslavia
dtypes: int64(6), object(9), uint8(204)
memory usage: 10.1+ MB


In [107]:
data.duplicated().sum()

24

In [109]:
data.drop_duplicates(keep=False, inplace=True)

In [110]:
data.duplicated().sum()

0

In [111]:
#removing dulpicate columns 
_, i = np.unique(data.columns, return_index=True) 
data=data.iloc[:, i] 

In [ ]:
data.info()

In [113]:

#Here our target variable is 'Income' with values as 1 or 0.  
#Separating our data into features dataset x and our target dataset y 
x=data.drop('salary',axis=1) 
y=data.salary

In [ ]:
x.head()

In [115]:
#Imputing missing values in our target variable 
y.fillna(y.mode()[0],inplace=True) 



/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [67]:
#Now splitting our dataset into test and train 
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)


#4° Máquina Preditiva

## XGBoost

In [116]:
#Applying xgboost

#The data is stored in a DMatrix object 
#label is used to define our outcome variable
dtrain=xgb.DMatrix(x_train,label=y_train)
dtest=xgb.DMatrix(x_test)



In [117]:

#setting parameters for xgboost
parameters={'max_depth':5,  
            'eta':1, 
            'silent':1,
            'objective':'binary:logistic',
            'eval_metric':'auc','learning_rate':.05}


#training our model 
num_round=50
from datetime import datetime 
start = datetime.now() 

xg=xgb.train(parameters,dtrain,num_round) 
stop = datetime.now()
#Execution time of the model 
execution_time_xgb = stop-start 
execution_time_xgb
#datetime.timedelta( , , ) representation => (days , seconds , microseconds) 



datetime.timedelta(0, 3, 373695)

In [118]:
#now predicting our model on test set 
ypred=xg.predict(dtest) 
ypred.size


9755

In [119]:

#Converting probabilities into 1 or 0  

for i in range(0,9755): 
    if ypred[i]>=.5:       # setting threshold to .5 
       ypred[i]=1 
    else: 
       ypred[i]=0  



#5° Avaliação da Máquina Preditiva com XGBoost

In [120]:
#calculating accuracy of our model 
from sklearn.metrics import accuracy_score 
accuracy_xgb = accuracy_score(y_test,ypred) 
accuracy_xgb


0.8551512045105074

## **Light_GBM**

## Hyperparâmetros

O **Light_GBM** usa divisão por folha em vez de divisão por profundidade, o que permite convergir muito mais rápido, mas também leva ao sobreajuste. Portanto, aqui está um guia rápido para ajustar os parâmetros no Light GBM.


### **Para** **melhor** **ajuste**


**num_leaves** : este parâmetro é usado para definir o número de folhas a serem formadas em uma árvore. Teoricamente, a relação entre num_leaves e max_depth é num_leaves = 2 ^ (max_depth). No entanto, esta não é uma boa estimativa no caso de GBM leve, uma vez que a divisão ocorre em relação à folha, e não à profundidade. Portanto, num_leaves definido deve ser menor que 2 ^ (max_depth), caso contrário, pode levar a sobreajuste. Light GBM não tem uma relação direta entre num_leaves e max_depth e, portanto, os dois não devem estar vinculados um ao outro.

**min_data_in_leaf** : É também um dos parâmetros importantes para lidar com overfitting. Definir seu valor menor pode causar sobreajuste e, portanto, deve ser definido de acordo. Seu valor deve ser de centenas a milhares de grandes conjuntos de dados.
max_depth : especifica a profundidade máxima ou nível até o qual a árvore pode crescer.
 


##**Para velocidade mais rápida**



**bagging_fraction** : é usado para realizar o ensacamento para resultados mais rápidos

feature_fraction : define a fração dos recursos a serem usados ​​em cada iteração
max_bin : O valor menor de max_bin pode economizar muito tempo, uma vez que agrupa os valores do recurso em caixas discretas, o que é computacionalmente barato.
 

##**Para melhor precisão**


* Use dados de treinamento maiores
* **num_leaves** : configurá-lo com um valor alto produz árvores mais profundas com maior precisão, mas leva ao sobreajuste. Portanto, seu valor mais alto não é preferido.
* **max_bin** : configurá-lo para valores altos tem efeito semelhante ao causado pelo aumento do valor de num_leaves e também torna mais lento nosso procedimento de treinamento.

In [121]:
# Light GBM
train_data=lgb.Dataset(x_train,label=y_train)
#setting parameters for lightgbm
param = {'num_leaves':150,         #número de folhas em uma árvore; padrão = 31; type = int
         'objective':'binary',     
         'max_depth':7,
         'learning_rate':.05,
         'max_bin':200}

param['metric'] = ['auc', 'binary_logloss']

#Here we have set max_depth in xgb and LightGBM to 7 to have a fair comparison between the two.

#training our model using light gbm
num_round=50
start=datetime.now()
lgbm=lgb.train(param,train_data,num_round)
stop=datetime.now()
#Execution time of the model
execution_time_lgbm = stop-start
execution_time_lgbm



datetime.timedelta(0, 0, 351131)

In [122]:
#predicting on test set
ypred2=lgbm.predict(x_test)
ypred2[0:5]  # showing first 5 predictions


array([0.02060976, 0.9400384 , 0.02314722, 0.02060976, 0.41985589])

In [123]:
ypred.size


9755

In [124]:
#converting probabilities into 0 or 1
for i in range(0,9755):
    if ypred2[i]>=.5:       # setting threshold to .5
       ypred2[i]=1
    else:  
       ypred2[i]=0


In [125]:
#calculating accuracy
accuracy_lgbm = accuracy_score(ypred2,y_test)
accuracy_lgbm



0.8592516658124039

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score
#calculating roc_auc_score for xgboost
auc_xgb =  roc_auc_score(y_test,ypred)
auc_xgb


# Comparando XGBoost vs Light_GBM

In [128]:
#calculating roc_auc_score for light gbm. 
auc_lgbm = roc_auc_score(y_test,ypred2)
auc_lgbm = comparison_dict = {'accuracy score':(accuracy_lgbm,accuracy_xgb),'auc score':(auc_lgbm,auc_xgb),'execution time':(execution_time_lgbm,execution_time_xgb)}


In [129]:
#Creating a dataframe ‘comparison_df’ for comparing the performance of Lightgbm and xgb. 
comparison_df = DataFrame(comparison_dict) 
comparison_df.index= ['LightGBM','xgboost'] 
comparison_df

,accuracy score,auc score,execution time
LightGBM,0.859252,0.756820,0 days 00:00:00.351131
xgboost,0.855151,0.752521,0 days 00:00:03.373695



### <font color='PURPLE'> Light_GBM é 9 (NOVE) Vezes mais Rápido !!!</font>



# **Resumão**

Nesta aula, tentei dar uma ideia intuitiva do Light GBM. Uma das desvantagens de usar esse algoritmo atualmente é sua base limitada de usuários - mas isso está mudando rapidamente. Este algoritmo, além de ser mais preciso e econômico do que o XGBOOST, tem seu uso limitado devido à menor documentação disponível.

No entanto, esse algoritmo mostrou resultados muito melhores e superou os algoritmos de aumento existentes. Eu recomendo fortemente que você implemente Light GBM sobre os outros algoritmos de boost e veja a diferença você mesmo.
Simbóra!

#Fim

## Valeu!

### #Links - Ciência dos Dados <a href="https://linktr.ee/cienciadosdados">https://linktr.ee/cienciadosdados</a>

### #YouTube - Mais Aulas como essa no YouTube <a href="https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s">https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s</a>

In [90]:
from IPython.core.display import HTML
HTML('<<iframe width="380" height="200" src="https://www.youtube.com/embed/FI3RlY9AaVs" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')